In [ ]:
#Import packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import numpy as np
import datetime as dt
from scipy.stats import kruskal

In [ ]:
#Extracting FOMC Data from Forbes [THIS WILL STOP WORKING IF THE WEBSITE CHANGES]

#Uncomment to show the entire dataframe
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

def fomcData():
    url = "https://www.forbes.com/advisor/investing/fed-funds-rate-history/"
    agent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=agent)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup.find_all('table', class_='foo-table')

    dataframes = []

    for table in tables:
        headers = [header.text for header in table.find_all("th")]

        rows = table.find_all("tr")[1:]

        table_data = []
        for row in rows:
            cells = row.find_all("td")
            row_data = []
            for cell in cells:
                text = cell.text.strip()
                if '%' in text and 'to' in text:
                    text = text.split('to')[-1].strip()
                row_data.append(text)
            table_data.append(row_data)

        df = pd.DataFrame(table_data, columns=headers)
        dataframes.append(df)

    all_data = pd.concat(dataframes, ignore_index=True)

    all_data['FOMC Meeting Date'] = all_data['FOMC Meeting Date'].str.replace('.', '', regex=True)
    all_data['FOMC Meeting Date'] = pd.to_datetime(all_data['FOMC Meeting Date'], errors='coerce')

    all_data.reset_index(drop=True, inplace=True)
    all_data = all_data.set_index('FOMC Meeting Date')
    all_data = all_data.sort_values(by='FOMC Meeting Date', ascending=True)

    if 'Change (bps)' in all_data.columns:
        mask = (all_data.index.year >= 1999) & (all_data.index.year <= 2001)
        all_data.loc[mask, 'Rate Change (bps)'] = all_data.loc[mask, 'Change (bps)']
        all_data = all_data.drop(columns=['Change (bps)'])

    all_data['Rate Change (bps)'] = all_data['Rate Change (bps)'].str.replace('+', '', regex=True).astype(float).astype('Int32', errors='ignore')
    all_data['Federal Funds Rate'] = all_data['Federal Funds Rate'].str.replace('%', '').astype(float) / 100

    #Getting all the FOMC Data to fill in 'No Change' Data
    allFOMC = pd.read_csv('fomc_dates.csv', index_col='Date')
    allFOMC.index = pd.to_datetime(allFOMC.index, format='%Y%m%d', errors='coerce')

    # Merge the DataFrames on their date index, using an outer join.
    merged_data = allFOMC.merge(all_data, how='left', left_index=True, right_index=True)

    # Fill NaN values in 'Rate Change (bps)' column with 0
    merged_data['Rate Change (bps)'].fillna(0, inplace=True)

    merged_data['Federal Funds Rate'].fillna(method='ffill', inplace=True)

    # Set a tolerance of 3 days
    tolerance = pd.Timedelta(days=3)

    # Use merge_asof to merge with tolerance
    merged_data = pd.merge_asof(allFOMC, all_data, left_index=True, right_index=True, direction='nearest', tolerance=tolerance)

    # Fill NaN values in 'Rate Change (bps)' column with 0
    merged_data['Rate Change (bps)'].fillna(0, inplace=True)
    merged_data['Federal Funds Rate'].fillna(method='ffill', inplace=True)

    start_date = pd.Timestamp('1990-11-13')
    merged_data = merged_data[merged_data.index >= start_date]

    return merged_data

#fomcData()

In [ ]:
#Classifying Data into 6 Quads
def classificationData():
    df = fomcData()

    def classify_action(val):
      if val > 0:
          return 'Tightening'
      elif val < 0:
          return 'Easing'
      else:
          return 'No Change'

    df['Action'] = df["Rate Change (bps)"].apply(classify_action)

    return df

#classificationData()

In [ ]:
fomc = classificationData()
fomc.index.names = ['Date']
#fomc

In [ ]:
#Getting the bond data
def getBond():
    endDate = dt.datetime.now()
    startDate = '1990-01-01'
    interval = '1d'
    bondData = yf.download('SHY',startDate ,endDate, interval=interval)
    bondData['Bond Close'] = bondData['Adj Close']
    bondData= bondData['Bond Close']
    bondData.index.names = ['Date']
    return bondData

#getBond()

In [ ]:
#Classifying Bond Data on Whether Bonds went Up or Down
def bondUpDown():
    bondData = getBond()
    df = fomc.merge(bondData, left_index=True, right_index=True, how='outer')

    missing_bond_data = df[df['Bond Close'].isna()]
    for idx, row in missing_bond_data.iterrows():
        next_idx = df.loc[idx:].dropna(subset=['Bond Close']).index.min()

        if pd.notna(next_idx):
            for col in fomc.columns:
                df.at[next_idx, col] = row[col]

    df = df.dropna(subset=['Bond Close'])
    df['Bonds PA'] = df['Bond Close'].diff().apply(lambda x: 'Up' if x > 0 else 'Down')

    return df

df = bondUpDown()

[*********************100%%**********************]  1 of 1 completed


In [ ]:
Asset = 'SPY'

#Getting ETF Data
def getAsset():
    endDate = dt.datetime.now()
    startDate = '1990-01-01'
    interval = '1d'
    assetData =  yf.download(Asset, startDate ,endDate, interval=interval)
    assetData['Asset Close'] = assetData['Adj Close']
    assetData= assetData['Asset Close']
    assetData.index.names = ['Date']
    return assetData

#getAsset()

In [ ]:
#Classifying Data based on Rolling Mean
def assetRM():
    assetData = getAsset()
    data = df.merge(assetData, on="Date")
    data['Asset RM'] = data["Asset Close"].rolling(20).mean()
    data['Mean Check'] = ['Above' if close > rolling_mean else 'Below' for close, rolling_mean in zip(data['Asset Close'], data['Asset RM'])]
    data['Sit'] =  data['Action'] + ' '  + data['Bonds PA'] + ' ' + data['Mean Check']
    return data

data = assetRM()

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# Calculate the number of rows for the in-sample and out-of-sample portions
total_rows = len(data)
in_sample_rows = int(0.7 * total_rows)
out_of_sample_rows = total_rows - in_sample_rows

# Split the DataFrame into in-sample and out-of-sample
in_sample = data.iloc[:in_sample_rows]
out_of_sample = data.iloc[in_sample_rows:]

# Now, 'in_sample' contains the first 70% of the data, and 'out_of_sample' contains the last 30%.
data = in_sample

In [ ]:
#This is the code for optimizing a particular scenario (DO NOT USE WHILE USING THE OUT-OF-SAMPLE DATASET)
def scenarioOptimzer():
    best_sharpe = -float('inf')
    best_entryDate = None
    best_exitDate = None
    best_positive_sharpe = -float('inf')
    best_negative_sharpe = float('inf')

    for entryDate in range(1, 10):
        for exitDate in range(2, 11):
            current_entryDate = entryDate
            current_exitDate = exitDate

            mask = (data['fomc'] == 1.0) & (~data['fomc'].isna())

            # Convert date-based entryDate and exitDate to row-based periods
            data['entryDate'] = data['Asset Close'].shift(-1 * current_entryDate)
            data['exitDate'] = data['Asset Close'].shift(-1 * current_exitDate)

            data['percent_change'] = (data['exitDate'] / data['entryDate'] - 1) * 100
            data['percent_change'] = data['percent_change'].where(mask)
            fomc_data = data[data['fomc'] == 1.0]
            num_days_per_meeting = current_exitDate - current_entryDate

            # Multiply by 8 (the average number of times the FOMC meets per year) for scaling
            scale = num_days_per_meeting * 8

            des = fomc_data.groupby(['Sit'])['percent_change'].describe()

            # Handle cases where standard deviation is zero or negative
            des['std'] = np.maximum(des['std'], 1e-6)  # Set a small positive minimum value for std
            des['Sharpe'] = (des['mean'] / des['std']) * np.sqrt(scale)

            final = des
            final.index.name = 'Sit'  # Set the index name to 'Sit'

            # Extract the Sharpe ratio for the specific group you're interested in
            current_sharpe = final.loc['Tightening Up Above']["Sharpe"]

            if abs(current_sharpe) > best_sharpe:
                best_sharpe = abs(current_sharpe)
                best_entryDate = current_entryDate
                best_exitDate = current_exitDate

            if current_sharpe > best_positive_sharpe:
                best_positive_sharpe = current_sharpe

            if current_sharpe < best_negative_sharpe:
                best_negative_sharpe = current_sharpe

    return best_entryDate, best_exitDate, best_sharpe, best_positive_sharpe, best_negative_sharpe


''' UNCOMMENT THIS TO RUN THE OPTIMZER FOR A PARTICULAR SCENARIO
best_entryDate, best_exitDate, best_sharpe, best_positive_sharpe, best_negative_sharpe = scenarioOptimzer()

print("Best Entry Date:", best_entryDate)
print("Best Exit Date:", best_exitDate)
print("Best Absolute Sharpe Ratio:", best_sharpe)
print("Best Positive Sharpe Ratio:", best_positive_sharpe)
print("Best Negative Sharpe Ratio:", best_negative_sharpe)
'''

' UNCOMMENT THIS TO RUN THE OPTIMZER FOR A PARTICULAR SCENARIO\nbest_entryDate, best_exitDate, best_sharpe, best_positive_sharpe, best_negative_sharpe = scenarioOptimzer()\n\nprint("Best Entry Date:", best_entryDate)\nprint("Best Exit Date:", best_exitDate)\nprint("Best Absolute Sharpe Ratio:", best_sharpe)\nprint("Best Positive Sharpe Ratio:", best_positive_sharpe)\nprint("Best Negative Sharpe Ratio:", best_negative_sharpe)\n'

In [ ]:
#Calculating the Sharpe based on a particular entry and exit date (Use Entry and Exit dates based on scenarioOptimzer() for a particular scenario)
def calculateReturn(entryDate, exitDate):
    mask = (data['fomc'] == 1.0) & (~data['fomc'].isna())

    data['entryDate'] = data['Asset Close'].shift(-1 * entryDate)
    data['exitDate'] = data['Asset Close'].shift(-1 * exitDate)

    data['percent_change'] = (data['exitDate'] / data['entryDate'] - 1) * 100
    data['percent_change'] = data['percent_change'].where(mask)
    fomc_data = data[data['fomc'] == 1.0]
    num_days_per_meeting = exitDate - entryDate

    # multiply by 8 (the average number of times the FOMC meets per year) for scaling
    scale = num_days_per_meeting * 8

    des = fomc_data.groupby(['Sit'])['percent_change'].describe()
    des['Sharpe'] = (des['mean']/des['std']) * np.sqrt(scale)

    final = des
    final['Sit'] = des.index
    return final

In [ ]:
#Outputting the Sharpe for a particular Entry and Exit Date
des = calculateReturn(1,9)
des

,count,mean,std,min,25%,50%,75%,max,Sharpe,Sit
Sit,,,,,,,,,,
Easing Down Above,2.0,1.696149,5.161059,-1.953270,-0.128561,1.696149,3.520859,5.345569,2.629149,Easing Down Above
Easing Down Below,1.0,1.801616,NaN,1.801616,1.801616,1.801616,1.801616,1.801616,NaN,Easing Down Below
Easing Up Above,3.0,-0.263170,1.268980,-1.426250,-0.939862,-0.453473,0.318371,1.090215,-1.659093,Easing Up Above
Easing Up Below,3.0,-1.851682,4.870713,-6.780873,-4.256705,-1.732537,0.612914,2.958366,-3.041332,Easing Up Below
No Change Down Above,30.0,0.585668,1.946757,-3.272343,-0.807348,0.687191,1.903239,4.560723,2.406743,No Change Down Above
No Change Down Below,17.0,0.777496,4.774494,-4.430944,-1.243443,-0.460781,2.632428,15.900718,1.302749,No Change Down Below
No Change Up Above,42.0,0.431063,1.785909,-4.351944,-0.813243,0.541279,1.526241,4.060872,1.930952,No Change Up Above
No Change Up Below,21.0,-1.887552,6.305062,-23.700258,-2.786290,-1.425288,2.386240,4.550257,-2.394967,No Change Up Below
Tightening Down Above,5.0,0.859037,1.160548,-0.479421,-0.223020,1.120569,1.824142,2.052915,5.921593,Tightening Down Above
